# Utils

In [10]:
# config = Config()
# edgeConnect = EdgeConnect(config)
# edgeConnect.train()

# %run inpaintgan.ipynb
import os.path
import sys
if "tkinter" not in sys.modules:
    import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from PIL import Image, ImageTk
from PIL import ImageDraw

import torch
import torchvision.transforms.functional as F
from torchinfo import summary



class ImageUploader(tk.Tk):
    def __init__(self):
        super().__init__()
        self.geometry("1600x1300")
        self.create_widgets()
        self.eraser_active = False  # Flag to keep track of eraser button state
        self.image_loaded = False  # Flag to keep track of whether an image is loaded
        self.mask = None
        self.mask_photo = None
        self.image = None
        self.photo = None
        self.edgeConfig = Config()
        self.edgeConfig.MODE = 4
        self.edgeConfig.MASK = 6
        self.edgeConnect = EdgeConnect(self.edgeConfig)
        self.edgeConnect.load()

        # Variables to store mouse coordinates
        self.start_x = None
        self.start_y = None

    def create_widgets(self):
        eraser_icon = Image.open(
            "screens/assets/eraser_icon.png")  # Replace "eraser_icon.png" with the actual file path of the icon image
        eraser_icon = eraser_icon.resize((32, 32))  # Resize the icon to desired dimensions
        self.eraser_image = ImageTk.PhotoImage(eraser_icon)

        self.eraser_button = tk.Button(self, image=self.eraser_image,  width=60, height=60, command=self.toggle_eraser, bd=1, highlightthickness=0, activebackground="gray")
        self.eraser_button.pack(side=tk.TOP, padx=10, pady=10, anchor=tk.N)

        self.image_label = tk.Label(self)
        self.image_label.pack(side=tk.LEFT, pady=10, padx=(0, 20), anchor=tk.NE)

        self.mask_label = tk.Label(self)
        self.mask_label.pack(side=tk.RIGHT, pady=10, padx=(20, 0), anchor=tk.NW)

        self.error_label = tk.Label(self, fg="red")
        self.error_label.pack(side=tk.BOTTOM, pady=10)

        self.fill_button = tk.Button(self, text="Fill Image", width=20, height=2, command=self.fill_image)
        self.fill_button.pack(side=tk.BOTTOM, pady=10, anchor=tk.S)

        self.upload_button = tk.Button(self, text="Upload Image", width=20, height=2, command=self.upload_image)
        self.upload_button.pack(side=tk.BOTTOM, pady=10, anchor=tk.S)

        self.name_label = tk.Label(self, text="")
        self.name_label.pack(side=tk.BOTTOM, pady=10, anchor=tk.S)

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=(("Image files", "*.jpg;*.jpeg;*.png"), ("All files", "*.*")))
        if file_path:
            self.image = Image.open(file_path)
            width, height = self.image.size
            if width > 600 or height > 800:
                ratio = min(600 / width, 800 / height)
                width = int(width * ratio)
                height = int(height * ratio)
                self.image = self.image.resize((width, height), Image.ANTIALIAS)
            new_size = (max(width, height), max(width, height))
            new_image = Image.new("RGBA", new_size, (0, 0, 0, 0))
            # Calculate the position to paste the original image without distortion
            paste_position = ((new_size[0] - self.image.width) // 2, (new_size[1] - self.image.height) // 2)
            # Paste the original image onto the new blank image
            new_image.paste(self.image, paste_position)
            self.image = new_image

            self.mask = Image.new("L", self.image.size)

            self.photo = ImageTk.PhotoImage(self.image)
            self.image_label.config(image=self.photo)
            self.image_label.image = self.photo

            self.mask_photo = ImageTk.PhotoImage(self.mask)
            self.mask_label.config(image=self.mask_photo)
            self.mask_label.image = self.mask_photo

            self.name_label.config(text="File Name: " + os.path.basename(file_path))
            self.error_label.config(text="")
            self.image_loaded = True

    def fill_image(self):
        if self.image_loaded:
            self.image.save('./image_mask.png')
            self.mask.save('./mask.png')
            # image_np = np.array(self.image)
            # # Convert the NumPy array to OpenCV format
            # image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
            # image_cv = self.resize(image_cv, 256, 256)
            # # Convert the mask image to grayscale
            # mask_image_gray = self.mask.convert("L")
            #
            # # Convert the grayscale image to a NumPy array
            # mask_array = np.array(mask_image_gray)
            # mask_array = self.resize(mask_array, 256, 256)
            # print(summary(self.edgeConnect.edge_model.get_submodule('generator'),(8, 3, 256, 256)))
            resultA = self.edgeConnect.fill_image(self.image.size[0])
            result = self.edgeConnect.postprocess(resultA)
            im = np.array(result.cpu()).astype(np.uint8).squeeze()
            im = Image.fromarray(im)
            im = im.resize(self.image.size, Image.ANTIALIAS)
            im.save('./output.png')
            self.photo = ImageTk.PhotoImage(im)
            self.image_label.config(image=self.photo)
        else:
            self.error_label.config(text="Error: No image uploaded")

    def toggle_eraser(self):
        self.eraser_active = not self.eraser_active  # Toggle eraser button state
        if self.eraser_active:
            # Change mouse cursor to eraser icon
            self.config(cursor="spraycan")
            self.eraser_button.configure(background="gray")
            self.image_label.bind("<Button-1>", self.start_drawing)
            self.image_label.bind("<B1-Motion>", self.erase_pixel)
            self.image_label.bind("<ButtonRelease-1>", self.stop_drawing)
        else:
            # Change mouse cursor back to normal and eraser button background to white
            self.config(cursor="")
            self.eraser_button.configure(background="white")
            self.image_label.unbind("<Button-1>")
            self.image_label.unbind("<B1-Motion>")
            self.image_label.unbind("<ButtonRelease-1>")

    def start_drawing(self, event):
        if self.image_loaded:
            self.start_x = event.x
            self.start_y = event.y

    def resize(self, img, height, width, centerCrop=True):
        imgh, imgw = img.shape[0:2]
        if centerCrop and imgh != imgw:
            # center crop
            side = np.minimum(imgh, imgw)
            j = (imgh - side) // 2
            i = (imgw - side) // 2
            img = img[j:j + side, i:i + side, ...]

        img = cv2.resize(img, (height, width))
        return img

    def stop_drawing(self, event):
        if self.image_loaded:
            self.start_x = None
            self.start_y = None

    def erase_pixel(self, event):
        if self.image_loaded and self.start_x is not None and self.start_y is not None:
            draw = ImageDraw.Draw(self.image)
            draw.line((self.start_x, self.start_y, event.x, event.y), fill="#ffffff", width=10)

            mask_draw = ImageDraw.Draw(self.mask)
            mask_draw.line((self.start_x, self.start_y, event.x, event.y), fill="#ffffff", width=10)

            self.mask_photo = ImageTk.PhotoImage(self.mask)
            self.mask_label.config(image=self.mask_photo)

            # Update self.photo with the modified image
            self.photo = ImageTk.PhotoImage(self.image)
            self.image_label.config(image=self.photo)

            # Update starting position to the current position
            self.start_x = event.x
            self.start_y = event.y
if __name__ == "__main__":
    gui = ImageUploader()
    gui.mainloop()
    # tkinter._test()generator(inputs

c:\users\moshpe\appdata\local\programs\python\python37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\users\moshpe\appdata\local\programs\python\python37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


in load EdgeModel /kaggle/working\EdgeModel_gen.pth
Loading EdgeModel generator...
in load InpaintingModel /kaggle/working\InpaintingModel_gen.pth
Loading InpaintingModel generator...


c:\users\moshpe\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:78: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
c:\users\moshpe\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:110: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


torch.Size([1, 3, 256, 256])


c:\users\moshpe\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:120: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.


torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])
